Components of the Entire Python3 program:

1. Find the numerical solutions, cn, for any "n" constants (represented by zn) and delta "d."

2. Define the polynomial as a list of coefficients.

3. Find close solutions. When we obtain solutions of the polynomial equations, we are only interested in the previous element of the sequence
   This part of the code returns the "closest" solutions.

4. Setting up a recursion.

5. Checking the solution is close to zero.

6. Checking that all solutions are different.

7. Define A(z1, z2) function. 

8. Define B1 and B2 function. 

9. Define a function for terms inside the summation. 

10. Define a function for the summation. 

11. We check the identity. 

12. Plotting the soltuion. 

In [2]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Import libraries for plotting (matplotlib) and advanced mathematical operations (using numpy)
import numpy as np
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')
import itertools as iter
import torch

In [4]:
#Part 1: Find the numerical solutions for any n constants

#FUNCTION DEFINITION: Accepts parameters delta "d" and *z as pointer allowing for any number of z values 
def compute_coefficient(d, *z):
    n = len(z)

    is_even = n % 2 == 0 #if there is no remainder when divided by two, is_even is set to true. Otherwise, set to false.

    #create an empty list to store the coefficients 
    coeffs = []

    for i in range(n):
        #initalize numerator and denominator for each element in the computation before updating the values in the second loop 
        numerator = 1
        denominator = 1
        
        for j in range(n):

            #compute the numerical solutions for the Bethe equations for the numerator and denominator 
            numerator *= (1 + z[i]*z[j] - d*z[i])
            denominator *= (1 + z[j]*z[i] - d*z[j])

        #append numerical solutions to the result and return as apart of the list
        if is_even:
            coeffs.append(-numerator/denominator) #when n is even, sign should be negative
        else:
             coeffs.append(numerator/denominator) #when n is odd, sign should be positive

    return coeffs

#FUNCTION CALL : Initialize list z and int d, store returned list and print 
print("\nTest examples for n = 2 and n = 4 cases : \n")

#Works for when 'z' contains two elements
z2 = [2, 3]
d2 = 4

coeffs2 = compute_coefficient(d2, *z2)
print(coeffs2)

#Check: Product of coefficients (all the elements in the list) should sum to one 
from functools import reduce
import operator
product2 = reduce(operator.mul, coeffs2)
print(product2)

print("\n")

#Works for when 'z' contains four elements
z4 = [2, 3, 4, 5]
d4 = 6

coeffs4 = compute_coefficient(d4, *z4)
print(coeffs4)

product4 = reduce(operator.mul, coeffs4)
print(product4)


Test examples for n = 2 and n = 4 cases : 

[-0.2, -5.0]
1.0


[-0.004784688995215311, -0.14285714285714285, -3.6666666666666665, -399.0]
1.0


In [47]:
compute_coefficient(0.1,2,3,4)

[1.0385282887886154, 0.9931139122315593, 0.96957766642806]

In [48]:

"""
Notes: For each coefficient, you want to create the list [1, 0, ..., 0, ci]
       Corresponds to z1L - c1 = 0, . . .,
       L roots depend on the ith coefficient 
       The solution of the polynomial will be stored in a Result list of length L with the roots 
       Once we have the solution above, we want to pick the closest solution 
"""

#Part 2: Define the polynomial as the list of coefficients 
def pol_coeffs(l, d, *z):

    c = compute_coefficient(d, *z) #c is a list that stores all the coefficients 
    #convert the list of n numbers to a list of n lists ; each number becomes its own list; each list is length l + 1
    #first element of the list will be 1, last will be ci, everything in between is zero 

    coeffs = [] #list of lists 
    results = [1]

    for i in c:
        
        for j in range (l-1):

            results.append(0)

        results.append(-i)
        coeffs.append(results) #attempt at appending for a specific coefficient (error message here, why?)
        results = [1] #creates a list of length l + 1  
        
    return coeffs

#test example from BAEsolutionsN2.ipynb 
print("\nTest example from BAE solutions.ipynb : \n")
l1 = 2
d1 = 0.1
z1 = [1j, -1j]
print(pol_coeffs(l1, d1, *z1))

print("\n")
l2 = 2
d2 = 0
z2 = [1, 1]
print(pol_coeffs(l2, d2, *z2))

print(pol_coeffs(5,0.1,2,3,4))


Test example from BAE solutions.ipynb : 

[[1, 0, (0.9950124688279302-0.09975062344139653j)], [1, 0, (0.9950124688279302+0.09975062344139653j)]]


[[1, 0, 1.0], [1, 0, 1.0]]
[[1, 0, 0, 0, 0, -1.0385282887886154], [1, 0, 0, 0, 0, -0.9931139122315593], [1, 0, 0, 0, 0, -0.96957766642806]]


In [54]:
#Part 3: Finding close solutions

"""
List of lists (for each sublists) -> array of roots -> pick the closest one 
Go back from n lists to n numbers 
"""

def close_sols(l, *zs, d, s):
    
    result = [] #empty list to store the closest solution for each coefficient  
    i = 0 
    #iterate over each sublist 
    for sublist in pol_coeffs(l, d, *zs):
        
        #calculate the roots
        roots = np.roots(sublist)
        
        #move to the previous element zi  
        roots = [x-s[i] for x in roots]  

        #find the solution with the minimum absolute value and add s
        min_sol = min(roots, key=np.abs) + s[i]  

        #results should be n numbers
        result.append(min_sol)

        i += 1

    return result  #Possible error of output not matching N = 2 case of E.Os of (0.049937616943892205+0.9987523388778446j) and (-0.04993761694389226+0.9987523388778448j)

cs_l = 2 
cs_z = [1j, -1j]
cs_d = 0.1
cs_s = [1j, 1j]

print("\nPrinting closed solutions when 'cs_l = 2, cs_z = [1j, -1j], cs_d = 0.1, and cs_s = 1j': \n")
print(close_sols(cs_l, *cs_z, d=cs_d, s=cs_s))

#print(close_sols(3,1,np.exp(2*np.pi*1j/3),np.exp(4*np.pi*1j/3),d=0.2,s=[1,1,1]))


Printing closed solutions when 'cs_l = 2, cs_z = [1j, -1j], cs_d = 0.1, and cs_s = 1j': 

[(0.049937616943892094+0.9987523388778446j), (-0.04993761694389226+0.9987523388778448j)]
[(1-9.790326495812671e-17j), (0.9835538253918807-0.18061526114090576j), (0.983553825391881+0.18061526114090581j)]


In [59]:
#Part 4: Setting up a recursion 

trials = 30

def sol_sys(l, d, *ks, trials):

    z = []
    list = pol_coeffs(l, 0, 0, 0)[0]
    sol = np.roots(list)

    for k in ks:
        z.append(sol[k])

    for _ in range(trials):
        z = close_sols(l, *z, d=d, s=z)

    return z

l = 5
s = 2
d = 0.1
ks = [1, 2, 3]
print("\nPrinting the solutions to the system:", sol_sys(l, d, *ks, trials=trials))

sol_sys(5,0.1,1,2,3,trials=30)



Printing the solutions to the system: [(-0.7660397456371132+0.6427932078858847j), (-0.7660397456371132-0.6427932078858847j), (1+0j)]


[(-0.7660397456371132+0.6427932078858847j),
 (-0.7660397456371132-0.6427932078858847j),
 (1+0j)]

In [8]:
#Part 5: Checking The Solution (Showing that all Solutions 'z' are Close to Zero) 
l = 5
d = 0.1
ks = [1, 4]
trials = 30

z = sol_sys(l, d, *ks, trials=trials)
c = compute_coefficient(*ks, d)

print("\nChecking solutions are close to zero:")

#print z[i] ** l minus coefficient[i] 
for sublist, coefficient in zip(z, c):
    for element in sublist:
        print(element**l - coefficient, ",")



Checking solutions are close to zero:


TypeError: 'numpy.complex128' object is not iterable

In [9]:
#Part 6: Checking the Solution (Showing that all Solutions 'z' are Different) 
l = 5
d = .1
trials = 300

#make sure it works for any number of ks ; make the list before passing each element k 
all_sol = [sol_sys(l, d, k1, k2, trials = trials) for k1 in range(l) for k2 in range(l) if k1 != k2]
print("\nPrinting all solutions: ", all_sol)

diff = [abs(x[0]-y[0])+abs(x[1]-y[1]) for x in all_sol for y in all_sol if x != y]
diff.sort()
print("\nPrinting different solutions: ", diff)


Printing all solutions:  [[(-0.9999004932239459-0.014106865367932153j), (-0.29556981885742134+0.9553211408634229j)], [(-0.9999004932239459+0.014106865367932153j), (-0.29556981885742134-0.9553211408634229j)], [(-0.9982161702461365-0.059703244963181246j), (0.8426665326923637+0.5384358036760085j)], [(-0.9982161702461365+0.059703244963181246j), (0.8426665326923637-0.5384358036760085j)], [(-0.29556981885742134+0.9553211408634229j), (-0.9999004932239459-0.014106865367932153j)], [(-0.2910397456371127+0.956710962861555j), (-0.2910397456371127-0.956710962861555j)], [(-0.2695024611955076+0.9629997006279724j), (0.7840690931242293+0.6206735512387721j)], [(-0.2863128246591293+0.9581361940954488j), (0.822766142365549-0.568380044492163j)], [(-0.29556981885742134-0.9553211408634229j), (-0.9999004932239459+0.014106865367932153j)], [(-0.2910397456371127-0.956710962861555j), (-0.2910397456371127+0.956710962861555j)], [(-0.2863128246591293-0.9581361940954488j), (0.822766142365549+0.568380044492163j)], [(

In [26]:
#Weight function
def W(zi,zj,d):
    w = (d*(1+zj**2-d*zj))/((1+zi*zj-d*zi)*(1+zi*zj-d*zj))
    return w

In [27]:
def create_M_matrix(z,d,l):
    n = len(z)
    # Create Laplacian weight matrix
    Lap = []
    for i in range(n):
        row = []
        for j in range(n):
            if i == j:
                diag = 0
                for k in range(n):
                    if i!=k:
                        diag += W(z[i],z[k],d)
                row.append(diag)
            if i!=j:
                row.append(-W(z[i],z[j],d))
        Lap.append(row)
    
  
    # Create lambda matrix
    Lam = np.zeros((n,n),dtype=complex)
    for i in range(n):
        Lam[i,i] = l/z[i]

    Lap = np.array(Lap,dtype=complex)
    # Create M matrix
    M = torch.from_numpy(Lam) + torch.tensor(Lap)

    return M

In [28]:
print(create_M_matrix([1,2],0.1,5))

tensor([[ 5.0591+0.j, -0.0591+0.j],
        [-0.0234+0.j,  2.5234+0.j]], dtype=torch.complex128)


In [29]:
# get C(z)
def get_coeff(z,d,l):
    # Create M matrix
    M = create_M_matrix(z,d,l)
    # Convert list to tensor
    new_z = torch.tensor(z)
    # Product of determinant of M matrix and (product of z's)/L^n
    coeff = (((torch.prod(new_z).item()))*torch.linalg.det(M)).item()
    return coeff


In [30]:
#Part 7: Checking the Initial Conditions 

# Product term furthest in the identity
def term1(z,perm,x,y):
    n = len(z)
    output = 1
    for i in range(n):
        output *= z[perm[i]-1]**(x[i]-y[perm[i]-1]-1)

    return output

# Sum over permutations in the symmetric group
def term2(z,x,y,d):
    n = len(z)
    # Identity permutation (ex. if n=3, id = (1 2 3))
    id = range(1,n+1)
    # Create set of permutations
    perms = iter.permutations(id)
    sum = 0

    for perm in perms:
        A = 1
        # A_sigma term
        for i in range(n):
            for j in range(i,n):
                if perm[i] > perm[j]:
                    A *= (-1)*(1+z[i]*z[j]-d*z[j])/(1+z[i]*z[j]-d*z[i])
        
        sum += A*term1(z,perm,x,y)
    
    return sum


def F_ic(all_sol,x,y,d,l):
    tot = 0

    for z in all_sol:
       # C(z)
       C = get_coeff(z,d,l)
       tot += (1/C)*term2(z,x,y,d) 
    
    return tot

    

In [35]:
#check initial condition
#N=2
k=2
l=5
d=0.2
trials = 30

#make sure it works for any number of ks ; make the list before passing each element k 
all_sol = [sol_sys(l, d, k1, k2, trials = trials) for k1 in range(l) for k2 in range(l) if k1 != k2]


y = list(range(0,k))
X = [(i+1, j+1) for i in range(l) for j in range(l) if i<j]

values = [F_ic(all_sol,x,y,d,l) for x in X ]

print("ic: ", y)
print(values)
print(sum(values))

ic:  [0, 1]
[(0.9999999999999999-6.938893903907228e-18j), (-3.469446951953614e-16+6.938893903907228e-18j), (3.400058012914542e-16-6.938893903907228e-18j), (7.979727989493313e-16+6.938893903907228e-18j), (-1.3530843112619095e-16+6.938893903907228e-18j), (-6.522560269672795e-16+2.0816681711721685e-17j), (6.453171330633722e-16-1.3877787807814457e-17j), (5.655198531684391e-16+0j), (-8.326672684688674e-16+0j), (-4.683753385137379e-16+6.938893903907228e-18j)]
(0.9999999999999998+2.0816681711721685e-17j)


In [43]:
#Check for N=3

k=3
l=5
d=0.1
trials = 100

#make sure it works for any number of ks ; make the list before passing each element k 
all_sol = [sol_sys(l, d, k1, k2, k3, trials = trials) for k1 in range(l) for k2 in range(l) for k3 in range(l) if (k1 != k2 and k2 != k3 and k1 != k3)]


y = list(range(0,k))
X = [(i+1, j+1, m+1) for i in range(l) for j in range(l) for m in range(l) if i<j<m]

values = [F_ic(all_sol,x,y,d,l) for x in X ]

print("ic: ", y)
print("all sols: ", all_sol)
print(values)
print(sum(values))

print(sol_sys(5, 0.1, 1, 2, 3,trials = 30))

ic:  [0, 1, 2]
all sols:  [[(-0.8021626023898193+0.5971056517294009j), (-0.8021626023898193+0.5971056517294009j), (-0.7952008389067593-0.6063461270610921j)], [(-0.8309475019311132+0.5563508326896291j), (-0.8309475019311132+0.5563508326896291j), (0.3809475019311122+0.9245966692414833j)], [(-0.82407017036398-0.5664877353626319j), (-0.7879305511761451+0.6157641159756354j), (0.3665304094587565-0.9304060720685331j)], [(-0.8021626023898186+0.5971056517294j), (-0.7952008389067593-0.6063461270610924j), (-0.8021626023898186+0.5971056517294j)], [(-0.82407017036398+0.5664877353626319j), (-0.7879305511761451-0.6157641159756354j), (0.3665304094587565+0.9304060720685331j)], [(-0.8309475019311132-0.5563508326896291j), (-0.8309475019311132-0.5563508326896291j), (0.3809475019311122-0.9245966692414833j)], [(-0.8309475019311119+0.5563508326896289j), (0.38094750193111265+0.9245966692414841j), (-0.8309475019311119+0.5563508326896289j)], [(-0.8240701703639792+0.5664877353626308j), (0.3665304094587566+0.9304

In [ ]:
#Part 8: Define B1 B2 Function


In [ ]:
#Part 9: Define a Function for terms inside the Summation

In [ ]:
#Part 10: Define a Function for the Summation

In [ ]:
#Part 11: We Check the Identity 

In [ ]:
#Part 12: Plotting the Solution